<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/7b_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines

sono uno strumento per fare quello che abbiamo fatto fino ad ora ovvero prendere il dataset, dividere le feature dal target, divido poi il dataset in 2 parti, una di train e uno di test, appico modifiche al dataset, andando a modificare i valori di alcuni colonne, e poi abbiamo allenato un modello sulla porzione di train e l'abbiamo testato sulla parte di test per vedere se le categorie predette sono simili da quelle reali (ovvero dai target), la pipeline permette di farlo. è una sequenza di trasformazioni che vengono applicate al dataset seguiute da un modello che verrà trainato sulla parte di train e testato su quelle di test. def una sequenze di trasformazioni e l'ultimo stesp è il modello da usare. le trasformazioni vengono eseguite in modo sequenziali. (comportamento solito). il risultato dell'ultima traformazionie viene dato in input per allenare il modello (fit)

* Pipelines sequentially apply **a list of transforms** and a **final estimator**.

    * Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods.
    * The final estimator only needs to implement fit.

* The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0) #solito divido il dataset in 2 parti train e validation

## Simple Pipeline

The simple pipeline is composed of the folloing steps:
- Transformation
    - Scaling values between 0 and 1
    - PCA (we keep 2 components)
- Estimator
    - Logistic Regression

definisco pipeline che applica trasformazioni e poi applico logistic regression per fare classificaion. le classificazioni sono lo scaling delle feature (min max scaler per fare in modo che i valori siano copresi tra 0 e 1) e poi applicao la pca per limitare il numer delle dimensioni-> partendo da 4 feature manteniamo 2 feature. queste 2 feature che derivano dalle 4 originali vegnono date in input alla logisitic regression che verrà prima allenato e poi farà la predizioni. creata la pipeline chiamo fit dando solo la parten di feature di train e poi il predict dando solo la parte del valid. la parte di predict daranno le classe predette che le andrò a confrontare con il target per vedere quanto è bravo il classificazione

### Transformation


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

# preprocessing_transformer = Pipeline(steps=[('normalize', Normalizer()),
#                                             ('scale_01', MinMaxScaler(feature_range=(0, 1))),
#                                             ('PCA', PCA(n_components=2))])
#definito la pipeline.di trasformazione fatta da 2 step->minmax scaler a cui do il nome che voglio io, il secondo step è il pca. abbiamo attributi delle etichette (stringhe) agli step
preprocessing_transformer = Pipeline(steps=[('scale_01', MinMaxScaler(feature_range=(0, 1))),
                                            ('PCA', PCA(n_components=2))])

### Estimator

In [4]:
from sklearn.linear_model import LogisticRegression
#l'estimetor è il classificatore e lo nizializzo come abbiamo sempre fatto

model = LogisticRegression(solver='lbfgs', multi_class='auto')

### Creating and evaluating the Pipeline

In [5]:
from sklearn.metrics import accuracy_score

#creo una nuova pipeline composta da 2 step, il primo step è a sua volta una pipelin ovvero quella di trasformazione mentre il secondo è il modello che abbiamo già inizializzato

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessing_transformer', preprocessing_transformer),
                               ('model', model)
                              ], verbose = True)

# my_pipeline = Pipeline(steps=[('scale_01', MinMaxScaler(feature_range=(0, 1))),
#                               ('PCA', PCA(n_components=2)),
#                               ('model', model)
#                              ], verbose = True)

In [6]:
# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train) #alleno il modello

[Pipeline]  (step 1 of 2) Processing preprocessing_transformer, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('preprocessing_transformer',
                 Pipeline(steps=[('scale_01', MinMaxScaler()),
                                 ('PCA', PCA(n_components=2))])),
                ('model', LogisticRegression(multi_class='auto'))],
         verbose=True)

In [7]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid) #predict a cui passo solo la parte di validation

# Evaluate the model
score = accuracy_score(y_valid, preds) #confronto le predizioni con il target del test e valuto come ha classificato. sto valutando tutta la pipeline. anche la trasformazioni

print('Accuracy Score:', score)

Accuracy Score: 0.8666666666666667


### Analyzing the transformation

In [8]:
X_train #4 feature per ogni entry del dataset (sepalo petalo altezza e larghezza)

array([[6.4, 3.1, 5.5, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [6.4, 2.8, 5.6, 2.2],
       [5.2, 2.7, 3.9, 1.4],
       [5.7, 3.8, 1.7, 0.3],
       [6. , 2.7, 5.1, 1.6],
       [5.9, 3. , 4.2, 1.5],
       [5.8, 2.6, 4. , 1.2],
       [6.8, 3. , 5.5, 2.1],
       [4.7, 3.2, 1.3, 0.2],
       [6.9, 3.1, 5.1, 2.3],
       [5. , 3.5, 1.6, 0.6],
       [5.4, 3.7, 1.5, 0.2],
       [5. , 2. , 3.5, 1. ],
       [6.5, 3. , 5.5, 1.8],
       [6.7, 3.3, 5.7, 2.5],
       [6. , 2.2, 5. , 1.5],
       [6.7, 2.5, 5.8, 1.8],
       [5.6, 2.5, 3.9, 1.1],
       [7.7, 3. , 6.1, 2.3],
       [6.3, 3.3, 4.7, 1.6],
       [5.5, 2.4, 3.8, 1.1],
       [6.3, 2.7, 4.9, 1.8],
       [6.3, 2.8, 5.1, 1.5],
       [4.9, 2.5, 4.5, 1.7],
       [6.3, 2.5, 5. , 1.9],
       [7. , 3.2, 4.7, 1.4],
       [6.5, 3. , 5.2, 2. ],
       [6. , 3.4, 4.5, 1.6],
       [4.8, 3.1, 1.6, 0.2],
       [5.8, 2.7, 5.1, 1.9],
       [5.6, 2.7, 4.2, 1.3],
       [5.6, 2

In [9]:
transformed_Dataset = preprocessing_transformer.fit_transform(X_train) #per vedere come è effettivamente trafsormato il dataset partendo dalle trasformazioni.  fit_transform permette di vedere il risultato della trasformazione

In [10]:
transformed_Dataset #il riuustlato è applciare prima il minmax scaler quindi partendo da 4 rimaniamo con 4 ma scalate e poi la pca che le porta a 2. è un array numpy il risultato

array([[ 3.92209763e-01,  4.84561075e-02],
       [ 9.04869356e-02, -8.95853266e-02],
       [-6.29042414e-01,  1.33227096e-01],
       [ 2.97394765e-01, -1.69364128e-02],
       [ 5.25821952e-01, -7.19394322e-02],
       [-8.81287220e-03, -2.16744469e-01],
       [-5.36677344e-01,  3.00755516e-01],
       [ 2.68771542e-01, -1.40752213e-01],
       [ 1.18143450e-01, -2.69059275e-02],
       [ 2.51091885e-02, -1.81673754e-01],
       [ 5.25692275e-01,  5.32019212e-02],
       [-6.94577142e-01, -3.59042800e-02],
       [ 5.43323143e-01,  1.04376017e-01],
       [-5.34922624e-01,  1.01994571e-01],
       [-6.15543324e-01,  2.31944792e-01],
       [-1.46419376e-01, -4.91909893e-01],
       [ 4.09356632e-01,  2.26626713e-02],
       [ 6.26859126e-01,  1.45225238e-01],
       [ 2.57239670e-01, -3.25774678e-01],
       [ 4.91330874e-01, -1.45418497e-01],
       [-3.11137441e-02, -2.39957062e-01],
       [ 7.51055857e-01,  1.48508623e-01],
       [ 2.30644832e-01,  1.25073917e-01],
       [-4.

In [12]:
type(transformed_Dataset)

numpy.ndarray

In [11]:
preprocessing_transformer.transform(X_valid) #applico soltato la trasformazione, restituisce le 2 feature (stesse trasformazioni applicate anche su validation)

array([[ 0.45431919, -0.13956513],
       [ 0.01613957, -0.305207  ],
       [-0.64028327,  0.4328635 ],
       [ 0.59569967,  0.06930586],
       [-0.64774017,  0.07208554],
       [ 0.61115082,  0.09432855],
       [-0.64749817,  0.11056931],
       [ 0.26200464,  0.09865541],
       [ 0.27341333, -0.00194264],
       [ 0.07738683, -0.07277631],
       [ 0.28573193, -0.16961619],
       [ 0.19957168,  0.10339402],
       [ 0.12542036, -0.07996038],
       [ 0.24300937, -0.03638314],
       [ 0.17388238, -0.04559015],
       [-0.70761088,  0.13831337],
       [ 0.16737484, -0.05646523],
       [ 0.03204393, -0.22204303],
       [-0.63470643, -0.1021321 ],
       [-0.59366964,  0.30640174],
       [ 0.30171174, -0.15420814],
       [ 0.11438335, -0.06601991],
       [-0.62885722,  0.04349897],
       [-0.70413094, -0.18523611],
       [ 0.30904546, -0.0790507 ],
       [-0.75940452,  0.10638345],
       [-0.56014656,  0.22594574],
       [ 0.11622089, -0.02718333],
       [-0.18340504,

## ColumnTransformer: Managing different kinds of transformers on different columns:
Extracted and extended from a kaggle.com tutorial

siccome ogni featue rappresenta quacosa di diverso magari voglio applicare alcuene trasformazioni ad alcune colonne ad altre ad altre. si usa il column trasformaer che permette di definire la trasformazione applicando la trasformazione alle colonne abc e l0altra per es a def

Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [14]:
dataset = pd.read_csv("melb_data.csv")
dataset.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [15]:
dataset.dtypes #ho sia feature numeriche che categoriche

,0
Suburb,object
Address,object
Rooms,int64
Type,object
Price,float64
Method,object
SellerG,object
Date,object
Distance,float64
Postcode,float64


In [16]:
dataset.shape

(13580, 21)

In [17]:
dataset.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [19]:
dataset['Price'] #è il nostro target, ciò che vogliamo predirre

,Price
0,1480000.0
1,1035000.0
2,1465000.0
3,850000.0
4,1600000.0
...,...
13575,1245000.0
13576,1031000.0
13577,1170000.0
13578,2500000.0


In [ ]:
#dataset = dataset[dataset['Price'].isnull()==False]

vglio applciare alcune trasformazioni alle feature categoriche ed altre alle feature numeriche. per fare ciò selezioni inizialmente quali sono le feature num e quali categoriche. uso il valore del d-type per farlo

In [20]:
columns = dataset.columns.to_list()
columns

['Suburb',
 'Address',
 'Rooms',
 'Type',
 'Price',
 'Method',
 'SellerG',
 'Date',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'CouncilArea',
 'Lattitude',
 'Longtitude',
 'Regionname',
 'Propertycount']

In [21]:
if 'Price' in columns:
  columns.remove('Price')

X = dataset[columns]
y = dataset['Price']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0) #splitto il dataset

We construct the full pipeline in three steps.
* Step 1: Define Preprocessing Steps
* Step 2: Define the Model
* Step 3: Create and Evaluate the Pipeline

### Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:
- imputes missing values in numerical data, and
- imputes missing values and applies a one-hot encoding to categorical data.

In [24]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and
                    X_train[cname].dtype == "object"]
                    #creo lista nomi di colonna dove il valore è categorica

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if
                X_train[cname].dtype in ['int64', 'float64']] #creo lista nomi di colonna dove il valore è numerico

In [25]:
categorical_cols

['Type', 'Method', 'Regionname']

In [26]:
numerical_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

definisco qual'è la traformazione per le colonne numeriche e quella per quelle categoriche. creo due trasformazioni diverse: la prima è una trasformazione fatta da un unico trasformatore-> sostituisce i valori nulli con la strategia del multi freqeutn, sostituisce il valore nullo con il valore più frequente per quella feature. la seconda trasforamzione è invece una pipeline di 2 step, il primo step è un simple imputer quindi fa la stessa cosa del primo, il secondo è un one hot encoder con la strategia di gestire il valore di gestire i valori sconosciuti (importante definirla!)
a questo punto creo quindi il columtrasoformer che applica il numeri cla trasformer alle colonne con i nomi dicolonna numerical:cols mentra applica il categorical:trasformer alle colonne con i nomi in categorical_cols.

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output = False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
       ('num', numerical_transformer, numerical_cols),
       ('cat', categorical_transformer, categorical_cols)
    ])

In [29]:
preprocessor
#l'input dato per ilt raining del modello saranno l'unione dei risultati delle due trasformazioni per darlo come input al training del modello

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='most_frequent'),
                                 ['Rooms', 'Distance', 'Postcode', 'Bedroom2',
                                  'Bathroom', 'Car', 'Landsize', 'BuildingArea',
                                  'YearBuilt', 'Lattitude', 'Longtitude',
                                  'Propertycount']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Type', 'Method', 'Regionname'])])

### Step 2: Define the Model

In [30]:
from sklearn.ensemble import RandomForestRegressor
#inizializzo un random forest come un modello

model = RandomForestRegressor(n_estimators=10, random_state=0)

### Step 3: Create and Evaluate the Pipeline

creo nuova pipelin uno è il column trasformer e l'altro è il modello

In [31]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
                              ('preprocessor', preprocessor),
                              ('model', model),
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Type', 'Method',
                                                   'Regionname'])])),
                ('model',
                 RandomForestRegressor(n_estimators=10, random_state=0))])

In [32]:
# Preprocessing of validation data, get predictions
#avendo fatto il fit della pipeline posso fare la predict che mi da le predizioni e le confronto con i  target
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

MAE: 173085.5993863525


### Parameter tuning

Setting parameters of the various steps is enabled by using their names and the parameter name separated by a ‘__’

con le pipeline posso fare anche il fine tuning degli iperparametri. come si fa? con il grid sarch come già visto, che non prenderà in input solo il modello della predizione ma tutta la pipeline, e tutti i valori degli iperparametri che voglio testare. costruisco lo spaizo degli iperparametri che voglio testare tramite un dizionario che ha però una particolarità. devo stare attento ai nomi degli iperparametri, siccome in questo caso ho una pipeline con diversi step duranti i quali ho diversi iperparametri.

voglio testare le diverse trasformazioni dei trasformatori. ho un simple imputer che può funzionare secondo diverse strategie, posso provare diverse strateegie per testare i valori nulli, non testo quindi solo gli iperparametri del random forset ma posso testare le strategie dei diversi trasformatori (nel one hot non c'è molto da testare ma il simple imputer avendo diverso strategie per gestire i valori nulli ha senso)

In [33]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'model__n_estimators': [1,5,10], #model è il nome che avevo dato io, devo dare quindi il nome__ mentre n_estimator è il nome del parametro da testare nella documentazione
    'preprocessor__num__strategy': ['most_frequent','constant'], #se voglio testare diverse trategie per i valroi num del simple imputer. di base ho preprocessor che è il nome della pipeline, all'interno di questa ce n'è un altra con all'interno uno step di nome "num" che metto dopo il __
    'preprocessor__cat__imputer__strategy': ['most_frequent','constant'], #per cat stesso ragionamento
}

gs_clf = GridSearchCV(my_pipeline, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

gs_clf.fit(X, y)
#gaurdando il grafichino qua sotto la nomeclatura è più semplice

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         ['Rooms',
                                                                          'Distance',
                                                                          'Postcode',
                                                                          'Bedroom2',
                                                                          'Bathroom',
                                                                          'Car',
                                                                          'Landsize',
                                                                          'BuildingArea',
                                                                          'YearBuilt',
                                                                          'Lattitude',
                                                                          'Longtitude',
                                                                          'Propertycount']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent...
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['Type',
                                                                          'Method',
                                                                          'Regionname'])])),
                                       ('model',
                                        RandomForestRegressor(n_estimators=10,
                                                              random_state=0))]),
             n_jobs=-1,
             param_grid={'model__n_estimators': [1, 5, 10],
                         'preprocessor__cat__imputer__strategy': ['most_frequent',
                                                                  'constant'],
                         'preprocessor__num__strategy': ['most_frequent',
                                                         'constant']},
             scoring='neg_mean_absolute_error')

abbiamo detto che facciamo il finetuning degli iperparametri, utilizziamo il gridsearch a cui diamo in input la pipeline e gli iperparametri da testare. il grid sarch ritorna siaa i valori dei parametri della configurazione milgiore che il best score che dipende da quanto inserito nel gridsarch, in questo caso abbiamo messo il negativo del mean absolute error quindi sarà - absolut error.

In [51]:
gs_clf.best_params_ #grid search ritorna sia i valori dei parametri della configurazione migliore

{'model__n_estimators': 10,
 'preprocessor__cat__imputer__strategy': 'most_frequent',
 'preprocessor__num__strategy': 'most_frequent'}

In [52]:
gs_clf.best_score_ #che ritorna il beast cscore che dipende da quello inserito nella parte di "scoring" del grid search

np.float64(-190165.69738200435)

Applying the pipeline to some attributes only

altra cosa importante del column trasformer che abbiamo detto che applica trasformazini diverse a prozioni diverse del dataset. se lo.inizializzo applicando la trasformazione min max scaler per es solo alle feature umeriche il risutlato del column trasformer saranno solo le feature numeriche trasformate con il num max scaler. tutto il dataset iniziale che contiene sia num che categoriche, applicando l columm trasformer che si applica solo alle feature numeriche, il risultato di questo column trasfomer saranno solo le feaure numeriche trasformate, quindi le feature categoriche il column trasfomer le sta buttando via, e questo starà l'input del modello. per ovviarlo devo dirgli di applicare la trasfformazioni alle feature numeriche e di lasciare tutto il resto invariato. occorre inserire il remainder = passthrought, cioè le numerical call le trasformi con il numerical trasformers, tutto il resto lasscialo invariato. si potrebbe anche dire di trsfromare quello che non è numerico con qualcosaltro, per farlo si usa per es reminder = one hot encoder, tutto ciò che non è una feature numerica trasformala con il one hot encoder.

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
        transformers=[('num', numerical_transformer, numerical_cols)],
        remainder='passthrough') #indica di lasciare tutto invariato. trassforma le numerical calls con il numerical trasformer mentre tutto il resto lascialo invariato

In [55]:
#This generates an error!

from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
                              ('preprocessor', preprocessor),
                              ('model', model),
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

ValueError: could not convert string to float: 'St Kilda'

Remainder with estimator

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')


# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
        transformers=[('num', numerical_transformer, numerical_cols)],
        remainder=OneHotEncoder(handle_unknown='ignore', sparse_output = False)) #tutto ciò che non è una feature numerica gestiscilo con un one hot encoder con l'ingor per gestire le categorie sconisciute

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
                              ('preprocessor', preprocessor),
                              ('model', model),
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

In [ ]:
my_pipeline

Alternative techniques to indicate columns

il column trasformer ha bisogno dei nomi di colonne per funzionare siccome gli indico di applicare la trasformazione X alla colonna A, necessario sapere il nome della colonna A quindi, se però applico prima delle trasformaizoni questo sarà un array numpy, quindi non avrò più i nomi di colonna, per ovviare a questo c'è un secondo modo per indicare i nomi di colonna. si utilizza l'x.train.columns.get_indexer dei nomi di colonna numerici. seleziono quindi gli indici di colnne e nell'array numpy ci sono gl indici di colonna e quindi posso utilizzare per selezionare i nomi di colonna numerica

In [56]:
X_train.columns.get_indexer(numerical_cols)

array([ 2,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 19])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')


# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, X_train.columns.get_indexer(numerical_cols))]) #anzi che dirgli di applicarlo alla numerical calls lo applica agli indici di quei nomi di colonna (stessa cosa ma per ovviare il problema)

In [ ]:
preprocessor.fit_transform(X_train)

Be carefull!

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Preprocessing for all dataset
simple=SimpleImputer(strategy='most_frequent')

# Preprocessing for numerical data
numerical_transformer = Normalizer()

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, numerical_cols)])

# Bundle preprocessing
preprocessing_transformer = Pipeline(steps=[('simple', simple),
                                            ('preprocessor', preprocessor)
                                            ])

In [ ]:
preprocessing_transformer

In [ ]:
preprocessing_transformer.fit_transform(X_train)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Preprocessing for all dataset
simple=SimpleImputer(strategy='most_frequent')

# Preprocessing for numerical data
numerical_transformer = Normalizer()

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, X_train.columns.get_indexer(numerical_cols))])

# Bundle preprocessing
preprocessing_transformer = Pipeline(steps=[('simple', simple),
                                            ('preprocessor', preprocessor)
                                            ])

In [ ]:
preprocessing_transformer.fit_transform(X_train)

## FeatureUnion: Applying multiple transformers in parallel

Concatenates results of multiple transformer objects.

This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

in una trsformazione classica abbiamo detto che le trasformazioni avvengono in sequenza, con il column trasformer applcio alcune trasformazioni ad una porzione di dataset che viene unita alle trasformazioni diverse ad un altra porzione di dataset e poi questo viene dato in input al modello. nella feature unioni ciò che succede è che ho tutto il dataset iniziale, applico alcune trasfromaioni a tutto questo dataset, questa trasformazioine restituisce delle feature trasformate, a tutto il dataset applico delle altre trasformazioni. e anche questa restituisce altro. questi due pezzi, questi due risultati di trasforamzioni differenti vengono uniti in un unico risultato che viene poi utilizzato per allenare e testare il modello. con il column applico due raformazioni diverse su due porzioni diverse. nella feature union l'inptu delle due traformazioni è l'interno dataset invece. le trasformazioni in questo caso non sono sequenziali ma in parallelo. input è il dataset iniziale sempre.

In [34]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [35]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [36]:
X.shape

(150, 4)

In [37]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

nella feature unionin voglio unire queste trasformzioni. la pca (parte da 4 feature e ottiene 2), in parallelo applicata una selection che è una selectkbest, partendo dalle 4 iniziali restituisce le 2 migliori e poi c'è uno scaler che partendo dalle 4 inziali feature restituisce le 4 feature scalate nel rang 0-1. se queste trasforamzioni sono applicate in parallelo alla fine della trasformazione avrò 8 feature (2 da pca, 2 da selection e 4 da scaler). input per ogni combinazione è il dataset iniziale, e i risutati vengono uniti

In [38]:
# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features were good, too?
selection = SelectKBest(k=2)

#Normalizing is always a good choice
scaler = MinMaxScaler(feature_range=(0, 1))

# Build estimator from PCA and Univariate selection:
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection), ("normal", scaler)])

In [40]:
combined_features

FeatureUnion(transformer_list=[('pca', PCA(n_components=2)),
                               ('univ_select', SelectKBest(k=2)),
                               ('normal', MinMaxScaler())])

In [39]:
pca.fit_transform(X).shape[1]

2

In [41]:
selection.fit_transform(X, y).shape[1]

2

In [42]:
scaler.fit_transform(X).shape[1]

4

In [43]:
# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)
print("Combined space has", X_features.shape[1], "features") #ho 8 feature, risultato di 3 trasformazioni in parallelo unite

Combined space has 8 features


In [44]:
X_features

array([[-2.68412563,  0.31939725,  1.4       , ...,  0.625     ,
         0.06779661,  0.04166667],
       [-2.71414169, -0.17700123,  1.4       , ...,  0.41666667,
         0.06779661,  0.04166667],
       [-2.88899057, -0.14494943,  1.3       , ...,  0.5       ,
         0.05084746,  0.04166667],
       ...,
       [ 1.76434572,  0.07885885,  5.2       , ...,  0.41666667,
         0.71186441,  0.79166667],
       [ 1.90094161,  0.11662796,  5.4       , ...,  0.58333333,
         0.74576271,  0.91666667],
       [ 1.39018886, -0.28266094,  5.1       , ...,  0.41666667,
         0.69491525,  0.70833333]])

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression() #utilizzo una logistic regression

#do in input la combinazione delle trasformazioni ad una logistic regression
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('combined_features', combined_features),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.1s


Pipeline(steps=[('combined_features',
                 FeatureUnion(transformer_list=[('pca', PCA(n_components=2)),
                                                ('univ_select',
                                                 SelectKBest(k=2)),
                                                ('normal', MinMaxScaler())])),
                ('model', LogisticRegression())],
         verbose=True)

In [46]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)
#con predict valuto quanto le trasformazioni si discostano dai valori relai
# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

Accuracy Score: 1.0


## FunctionTransformer: Constructs a transformer from an arbitrary callable.

Concatenates results of multiple transformer objects.

A FunctionTransformer forwards its X (and optionally y) arguments to a user-defined function or function object and returns the result of this function.

trasformazioni personalizzate. tutti i trasformatori visto fino ad ora possono essere inseriti come step delle pipeline perchè inplementano fit e fit trasform che servono per essere inseriti in una pipeline. posso decidere di voler trasformazioni peronalizzate. per fare questo devo usare le function transofremr permettono di costruire un trasformatore con funzioni di fit e fit trasform per costruire trasformazioni personalizzate.
nell'esempio qua sotto il function trasformer viene utilizzato definendo la pipeline in 2 step. il secondo è il calssico simple imputer mentre il primo è il function trasformer che è una funzione personalizzata. applcia a tutto il dataset la trasformazione columns_num che ho definito io -> seleziona le colonne numeriche e restituisce le colonne numeriche, il primo step semplicemente restituisce solo le colonne numeriche del dataset. seleziona le feature numeriche del dataset e ritorna solo queste. quindi il simple imputer viene applicato solo alla feature numeriche siccome le trasformazioni sono sequenziali. occorre quindi inizalizzare il mdoello randomforest in questo caso, ora inializzo la seconda pipeline che unisce la trasformazione al modello. si applica quindi il fit per allenare il modello

In [47]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor

dataset = pd.read_csv("melb_data.csv")
columns = dataset.columns.to_list()
columns.remove('Price')

X = dataset[columns]
y = dataset['Price']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

#Selecting the numerical columns
def columns_num(X):
    numerical_cols = [cname for cname in X.columns if  X[cname].dtype in ['int64', 'float64']]
    return X.loc[:,numerical_cols]

fill_na_transformer = Pipeline(steps=[ ('drop_cols', FunctionTransformer(columns_num, validate=False)), #validate = false fondamentale per non trasformare df in array numpy in modo da usare nomi di colonna nella funzione, guarda sotto per spiegazioine
                                       ('fill_na', SimpleImputer(strategy='most_frequent')) ], verbose=True)


model = RandomForestRegressor(n_estimators=100, random_state=0)



# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', fill_na_transformer),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing drop_cols, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing fill_na, total=   0.0s


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('drop_cols',
                                  FunctionTransformer(func=<function columns_num at 0x79c79f58f7e0>)),
                                 ('fill_na',
                                  SimpleImputer(strategy='most_frequent'))],
                          verbose=True)),
                ('model', RandomForestRegressor(random_state=0))])

In [48]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 169836.96078617012


nel momento in cui applico una trasforamzione abbiamo detto che il risultato di questa è un array numpy quindi anche partendo da un dataframe dopo la prima trasformazione non avrò più il dataframe ma l'array numpy. per cui perdo i nomi di colonna. per ovviare a questo provlema si utilizza il validate=false nel function trasformer. partendo da un dataframe e utilizzo nel function trasformer il dataframe non viene validato e quindi rimane il dataframe con il nome di colonna, è il motivo per cui colmns num può usare i nomi di colonna per fare ciò che deve. con validate = true chiamando il fit trasform si generra un errore, perchè non esistono i nomi di colonna quindi la traformazione non si esegue come vogliamo e quindi si genera un errore perchè la pipeline non è in grado di convertire la stringa in float per far il trasform secondo la pipeline. se ho bisogno dei nomi di colonna per fare trasformazioni devo utilizzare il validate = false, in questo modo il dataframe non viene trasformato in array numpy e quindi posso usare i nomi di colonna per applicare alcune traformazioni solo a determinate colonne

In [49]:
fill_na_transformer = Pipeline(steps=[ ('drop_cols', FunctionTransformer(columns_num, validate=True)),
                                       ('fill_na', SimpleImputer(strategy='most_frequent')) ])

In [50]:
fill_na_transformer.fit_transform(X)

ValueError: could not convert string to float: 'Abbotsford'

In [ ]:
X

In [ ]:
columns_num(X)